In [22]:
import pandas as pd
import numpy as np
import math

### Step 0
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

# Data wrangling

In [23]:
energy = pd.read_excel('datasets\data\Energy Indicators.xls', index_col=None, skiprows=17, skipfooter=38 )#read data

In [24]:
energy.head() #diplay table

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Petajoules,Gigajoules,%
0,NaN,Afghanistan,Afghanistan,321,10,78.669280
1,NaN,Albania,Albania,102,35,100.000000
2,NaN,Algeria,Algeria,1959,51,0.551010
3,NaN,American Samoa,American Samoa,...,...,0.641026
4,NaN,Andorra,Andorra,9,121,88.695650


In [25]:
energy = energy.drop(energy.columns[[0,1]], axis=1) #dropping first two column

In [26]:
energy.head() #diplay table

,Unnamed: 2,Petajoules,Gigajoules,%
0,Afghanistan,321,10,78.669280
1,Albania,102,35,100.000000
2,Algeria,1959,51,0.551010
3,American Samoa,...,...,0.641026
4,Andorra,9,121,88.695650


In [27]:
old_names = ["Unnamed: 2", "Petajoules", "Gigajoules" ,"%"] #Renaming columns
new_names = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
for i in range(4) :
    energy.rename(columns = {old_names[i]:new_names[i]},inplace = True)
energy.head() #checking result

,Country,Energy Supply,Energy Supply per Capita,% Renewable
0,Afghanistan,321,10,78.669280
1,Albania,102,35,100.000000
2,Algeria,1959,51,0.551010
3,American Samoa,...,...,0.641026
4,Andorra,9,121,88.695650


In [28]:
for i in ['Energy Supply']:
    energy['Energy Supply'] = energy['Energy Supply'].apply(lambda i: np.nan if i== "..." else i) #change ... to NaN
    energy['Energy Supply'] = energy['Energy Supply']*1000000 #Convert `Energy Supply` to gigajoules

In [29]:
Country_no = []
for i in energy["Country"]:
    i = i.rstrip('0123456789') #stripping numbers
    i = i.split("(")[0].strip() #stripping parethesis
    Country_no.append(i)
energy["Country"] = Country_no 

In [30]:
country_names = {"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"}
def country_replace(list, column):
    for i in column: #Replacing the list of countries
        column = column.apply(lambda i: list[i] if i in list else i)
    return column
energy['Country'] = country_replace(list=country_names, column=energy['Country'])

### Step 1
<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

In [31]:
GDP = pd.read_csv('datasets\data\world_bank.csv', skiprows=4) #Reading CSV
country_names2 = {"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"} #Replacing list of country names
GDP['Country Name'] = country_replace(list=country_names2, column=GDP['Country Name'])

In [32]:
GDP.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN
1,Andorra,AND,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN
2,Afghanistan,AFG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.030523e+10,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10
3,Angola,AGO,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,5.581103e+10,6.842044e+10,7.787420e+10,7.975320e+10,8.247091e+10,8.570262e+10,9.012096e+10,9.626143e+10,1.008863e+11,1.039106e+11
4,Albania,ALB,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,9.771760e+09,1.034829e+10,1.112752e+10,1.150029e+10,1.192695e+10,1.223109e+10,1.240477e+10,1.254247e+10,1.279331e+10,1.312082e+10


### Step 2
Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

In [33]:
ScimEn = pd.read_excel('datasets\data\scimagojr-3.xlsx', index_col=False) #Read scimagojr-3.xlsx

In [34]:
ScimEn.head()

,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,127050,126767,597237,411683,4.70,138
1,2,United States,96661,94747,792274,265436,8.20,230
2,3,Japan,30504,30287,223024,61554,7.31,134
3,4,United Kingdom,20944,20357,206091,37874,9.84,139
4,5,Russian Federation,18534,18301,34266,12422,1.85,57


### Step 3
Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This step should yeild a DataFrame with 20 columns and 15 entries.*

In [35]:
df = pd.merge(ScimEn, energy, how="left", on=["Country", "Country"]).iloc[0:15] #Merge ScimEn(first 15 row) and energy dataframes

In [36]:
mergeddf = df.merge(GDP[['2006', '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015', 'Country Name']], how = 'left', #Merge all dataframes with GDP specified columns
                left_on = 'Country', right_on = 'Country Name').drop(columns = ['Country Name'])
mergeddf = mergeddf.set_index('Country') #Change index 
mergeddf

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286,11.570980,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149,10.232820,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124,10.600470,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


### Step 4
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This step should yield a single number.*

In [37]:
num_before = pd.merge(ScimEn, energy, how="left", on=["Country", "Country"]).shape[0]
num_after = df.shape[0]
num_before - num_after

176

### Step 5

#### Answer the following questions in the context of only the top 15 countries by Scimagojr Rank 


What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This step should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [38]:
mergeddf_15 = mergeddf[['2006', '2007', '2008','2009', '2010', '2011', '2012', '2013', '2014', '2015']]
avgGDP = []
for indx, ele in enumerate(mergeddf_15.values):
    ele = ele[~np.isnan(ele)]
    avg = np.average(ele if len(ele) > 0 else 0 )
    obj = {'Country': mergeddf.index[indx],'Average': avg}
    avgGDP.append(obj)
avgGDP = sorted(avgGDP, key=lambda d: d['Average'], reverse=True)
avgGDP 

[{'Country': 'United States', 'Average': 15364344302990.0},
 {'Country': 'China', 'Average': 6348608932836.1},
 {'Country': 'Japan', 'Average': 5542207638235.176},
 {'Country': 'Germany', 'Average': 3493025339072.8477},
 {'Country': 'France', 'Average': 2681724635761.589},
 {'Country': 'United Kingdom', 'Average': 2487906661418.4175},
 {'Country': 'Brazil', 'Average': 2189794143774.905},
 {'Country': 'Italy', 'Average': 2120175089933.7761},
 {'Country': 'India', 'Average': 1769297396603.86},
 {'Country': 'Canada', 'Average': 1660647466307.512},
 {'Country': 'Russian Federation', 'Average': 1565459478480.661},
 {'Country': 'Spain', 'Average': 1418078278145.694},
 {'Country': 'Australia', 'Average': 1164042729991.427},
 {'Country': 'Iran', 'Average': 444155754051.095},
 {'Country': 'South Korea', 'Average': 0.0}]

### Step  6
What is the mean `Energy Supply per Capita`?

*This step should return a single number.*

In [39]:
mean_energy_supply = np.mean(mergeddf['Energy Supply per Capita'])
mean_energy_supply

157.6

### Step 7
What country has the maximum % Renewable and what is the percentage?

*This step should return a tuple with the name of the country and the percentage.*

In [40]:
max_renew = np.max(mergeddf['% Renewable'])
for indx, i in enumerate(mergeddf[['% Renewable']].values):
    if i == max_renew:
        mxcountry = mergeddf.index[indx]
mxcountry, max_renew

('Brazil', 69.64803)

### Step 8
Create a new column that is the ratio of Self-Citations to Total Citations. What is the maximum value for this new column, and what country has the highest ratio?

This step should return a tuple with the name of the country and the ratio.

In [41]:
ScimEn['Citation Ratio'] = ScimEn['Self-citations']/ScimEn['Citations per document']
MaxCitationRatio = ScimEn['Citation Ratio'].max()
for indx, i in enumerate(ScimEn[['Citation Ratio']].values):
    # print(indx, i)
    if i == MaxCitationRatio:
        MxRTcountry = ScimEn.loc[indx, 'Country']
MaxCitationRatio, MxRTcountry

(87592.12765957447, 'China')

### Step 9
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This step should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [44]:
MedianCntry = np.median(mergeddf['% Renewable'])
for indx, i in enumerate(mergeddf['% Renewable']):
    if i >= MedianCntry:
        Country_name = mergeddf.index[indx]
        mergeddf.loc[[Country_name], 'HighRenew'] = 1
    else:
        Country_name = mergeddf.index[indx]
        mergeddf.loc[Country_name, 'HighRenew'] = 0
mergeddf['HighRenew'] = mergeddf['HighRenew'].astype(int) #correcting data type
mergeddf['HighRenew']

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int32

### Step 10
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [43]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
continent ={'Asia':[], "Australia":[], "Europe":[], "North America":[],"South America":[]}

NewDf = pd.DataFrame(columns=['size', 'sum', 'mean', 'std'], index=['Asia', 'Australia', 'Europe', 'North America', 'South America']).fillna('')
for indx, i in enumerate(mergeddf.index):
        Numerator = mergeddf.loc[i, 'Energy Supply']
        denominator = mergeddf.loc[i, 'Energy Supply per Capita']
        population = math.floor(Numerator/denominator)
        continent[ContinentDict[i]].append(population)
for i in NewDf.index:
        NewDf.loc[[i], 'size'] = len(continent[i])
        NewDf.loc[[i], 'sum'] =np.int64(continent[i]).sum()
        NewDf.loc[[i], 'mean'] =np.mean(continent[i])
        NewDf.loc[[i], 'std'] =np.std(continent[i])

NewDf

,size,sum,mean,std
Asia,5,2898666384,579733276.8,607403616.914446
Australia,1,23316017,23316017.0,0.0
Europe,6,457929664,76321610.666667,31628848.19392
North America,2,352855248,176427624.0,141187760.0
South America,1,205915254,205915254.0,0.0
